<h1 align="center"><b>Exercise on Games</b></h1>

---

Import all the necessary packages

In [106]:
import random
import numpy as np
import time
from collections import namedtuple

Class Game, which is a super class of `TicTacToe`

In [107]:
class Game:
    """Defines a `Game` data structure
    
    Attributes:
        `self.initial`: the initial state of the game"""
    def __init__(self, initial_state):
        """Initialize a new game, and store into `self.initial` the `initial_state` given as input
        
        Parameters:
            `initial_state`: the state on which the `Game` is based"""
        self.initial = initial_state

    def play(self, players):
        """Plays the next step of the game, given as input two players
        
        Parameters:
            `players`: a list of players"""
        state = self.initial    # Set the current state as the initial state of the game
        while True:     # While true...
            for player in players:  # ...for each player in the list of players...
                #print(f"→ The player is {player}")
                if self.is_terminal(state):     # ...if the state would make the game end...
                    return  # ...then break from the loop
                move = player(self, state)  # otherwise get the next move
                state = self.result(state, move)    # determine the next state, given a state and a move
                self.display(state)     # display such state

    def actions(self, state):
        raise NotImplementedError

    def result(self, state, move):
        raise NotImplementedError

    def utility(self, state, player):
        raise NotImplementedError

    def is_terminal(self, state):
        return NotImplementedError

    def display(self, state):
        return NotImplementedError

GameState = namedtuple('GameState', ['to_move', 'utility', 'board', 'moves'])

In [108]:
def evaluation_function(board: dict):
    """Evaluates the current situation, and returns the difference between the places on """
    # Board: dict((pos_x, pos_y): "player", ...)
    npbd = np.ones((3, 3))
    for key in board.keys():
        #print(f"Key: {key} - Value: {value}")
        if board[key] == "O":
            npbd[key[0]-1][key[1]-1] = 0
    print(npbd)


Class tictactoe

In [109]:
class TicTacToe(Game):
    """A `TicTacToe` game, which is a subclass of the `Game` class"""
    def __init__(self, h=3, v=3, k=3):
        """Initialize TicTacToe with board size and winning condition
        
        Parameters
            `h` (normally `3`): height of the board;
            `v` (normally `3`): width of the board;
            `k` (normally `3`): ???
        """
        super().__init__(GameState(to_move='X', utility=0, board={},
                                   moves=self._all_possible_moves(h, v)))   # The game always starts with X
        self.h = h
        self.v = v
        self.k = k

    def _all_possible_moves(self, h, v):
        """Generate all possible moves on the given board size."""
        return [(x, y) for x in range(1, h + 1) for y in range(1, v + 1)]

    def actions(self, state):
        """Returns the moves of a state
        
        Parameters:
            `state`: a state of which we want to know the actions"""
        return state.moves

    def result(self, state, move):
        """Does smth"""
        if move not in state.moves: # If the move is not in the 
            return state
        board = state.board.copy()
        board[move] = state.to_move
        moves = list(state.moves)
        moves.remove(move)
        next_player = 'O' if state.to_move == 'X' else 'X'
        return GameState(to_move=next_player,
                        utility=self.compute_utility(board, move, state.to_move),
                        board=board, moves=moves)

    def utility(self, state, player):
        return state.utility if player == 'X' else -state.utility

    def is_terminal(self, state):
        return state.utility != 0 or len(state.moves) == 0

    def display(self, state):
        board = state.board
        for x in range(1, self.h + 1):
            for y in range(1, self.v + 1):
                print(board.get((x, y), '.'), end=' ')
            print()

    def compute_utility(self, board, move, player):
        """If 'X' wins with this move, return 1; if 'O' wins return -1; else return 0."""
        if (self.k_in_row(board, move, player, (0, 1)) or
                self.k_in_row(board, move, player, (1, 0)) or
                self.k_in_row(board, move, player, (1, -1)) or
                self.k_in_row(board, move, player, (1, 1))):
            return +1 if player == 'X' else - 1
        else:
            return 0

    def k_in_row(self, board, move, player, delta_x_y):
        """Return true if there is a line through move on board for player."""
        (delta_x, delta_y) = delta_x_y
        x, y = move
        n = 0  # n is number of moves in row
        while board.get((x, y)) == player:
            n += 1
            x, y = x + delta_x, y + delta_y
        x, y = move
        while board.get((x, y)) == player:
            n += 1
            x, y = x - delta_x, y - delta_y
        n -= 1  # Because we counted move itself twice
        return n >= self.k

In [110]:
MAX, MIN = np.inf, -np.inf

def minmax_search(game, state):
    player = state.to_move

    def max_value(state):
        evaluation_function(state.board)
        if game.is_terminal(state):
            return game.utility(state, player)
        v = MIN
        for successor in game.actions(state):
            v = max(v, min_value(game.result(state, successor)))
        return v
    
    def min_value(state):
        evaluation_function(state.board)
        if game.is_terminal(state):
            return game.utility(state, player)
        v = MAX
        for successor in game.actions(state):
            v = min(v, max_value(game.result(state, successor)))
        return v
    
    print(f"\n→ Turn of {player}, the available actions are {game.actions(state)}")
    print(state.board)
    evaluation_function(state.board)
    return max(game.actions(state), key=lambda a: min_value(game.result(state, a)))

def random_player(game, state):
    return random.choice(list(game.actions(state)))

def player(search_algorithm):
    return lambda game, state: search_algorithm(game, state)

In [111]:
TicTacToe().play([player(minmax_search), random_player])


→ Turn of X, the available actions are [(1, 1), (1, 2), (1, 3), (2, 1), (2, 2), (2, 3), (3, 1), (3, 2), (3, 3)]
{}
